In [ ]:

#@title Importing required packages
import sys
import os
import re
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%load_ext tensorboard

In [ ]:
#@title Fetching PAMAP
# PAMAP
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00231/PAMAP2_Dataset.zip
! unzip PAMAP2_Dataset.zip

In [ ]:
! ls

In [ ]:
#@title Fetching MHealth
# MHEALTH
! wget http://archive.ics.uci.edu/ml/machine-learning-databases/00319/MHEALTHDATASET.zip
! unzip MHEALTHDATASET.zip

In [ ]:
#@title Packages
!apt install poppler-utils

In [ ]:

#@title MHealth Dataset Readme
! cat MHEALTHDATASET/README.txt

In [ ]:

#@title -

!pdftotext PAMAP2_Dataset/readme.pdf

In [ ]:
#@title PAMAP2 Dataset Readme
!cat PAMAP2_Dataset/readme.txt

In [ ]:
#@title Inspecting MHealth log File
!head -n 10 MHEALTHDATASET/mHealth_subject1.log

In [ ]:
#@title Inspecting PAMAP2 file
!head -n 10 PAMAP2_Dataset/Protocol/subject101.dat

In [ ]:

#@title MHealth Files List
for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    print(i)

In [ ]:
#@title Converting one subject to DataFrame
mHealth = pd.read_csv('MHEALTHDATASET/mHealth_subject3.log', header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"})
mHealth.head(5)

In [ ]:
sns.pairplot(mHealth)

EDA


In [ ]:
#@title Describe MHealth Dataset One Subject
mHealth.describe().T

In [ ]:
#@title Describe MHealth for all
all = mHealth[mHealth["Output"] == 14].copy()

for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    all = all.append(pd.read_csv('MHEALTHDATASET/'+i, header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"}))


description = all.describe()

description.T

In [ ]:
description["alX"]["max"]

mhealth dataset

In [ ]:
#@title Size of dataset for various subjects
for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    print(pd.read_csv('MHEALTHDATASET/'+i, header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"}).shape)

In [ ]:
temp.isnull().values.any()

In [ ]:
#@title Importing and preprocessing dataset
SAMPLES_PER_ACTIVITY=119  #@param { "type": "integer" }

Activities = [
    "Nothing",
    "Standing",
    "Sitting and Relaxing",
    "Lying Down",
    "Walking",
    "Climbing Stairs",
    "Waist bends forward",
    "Frontal Elevation of Arms",
    "Knees Bending (Crouching)",
    "Cycling",
    "Jogging",
    "Running",
    "Jump Front & Back"
    ]


NUM_ACTIVITY=len(Activities)
X = []
Y = []

FEATURES=["alX", "alY", "alZ", "glX", "glY", "glZ", "arX", "arY", "arZ", "grX", "grY", "grZ"]
ONE_HOT_ENCODED_ACTIVITY = np.eye(NUM_ACTIVITY)
for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    temp = pd.read_csv('MHEALTHDATASET/'+i, header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"})
    for j in range(NUM_ACTIVITY):
        output = ONE_HOT_ENCODED_ACTIVITY[j]
        df =np.array(temp[temp["Output"] == j].copy())
        num_recordings = int(df.shape[0]/SAMPLES_PER_ACTIVITY)
        print(f"\tThere are {num_recordings} recordings of the {Activities[j]} activity.")
        for k in range(num_recordings):
          tensor = []
          for l in range(SAMPLES_PER_ACTIVITY):
            index = k * SAMPLES_PER_ACTIVITY + l

            tensori = []
            for m,feature in enumerate(FEATURES):
              tensori.append(df[index,m] + (description[feature]["mean"]+3*description[feature]["std"]) / (3*description[feature]["std"]))
            
            tensor.append(tensori)
          X.append(tensor)
          Y.append(output)

X = np.array(X)
Y = np.array(Y)


print("Data set parsing and preparation complete.")

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], X.shape[2], 1))

In [ ]:

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [ ]:

from tensorflow.keras import layers, Sequential, Input

from sklearn.model_selection import train_test_split

In [ ]:
input_shape=X.shape[1:]

X.shape

In [ ]:
grumodel = Sequential(
    [
     Input(shape=input_shape),
     layers.Conv2D(64, (3,3)),
     layers.MaxPool2D((2,2)),
     layers.BatchNormalization(),  
     layers.Reshape((-1, 64)),
     layers.GRU(15),
     layers.Dense(128, activation="relu"),
     layers.Dense(13, activation="sigmoid"),

    ]

)

In [ ]:
grumodel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
import datetime

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
grumodel.fit(x=X_train, 
          y=Y_train, 
          epochs=20, 
          validation_data=(X_test, Y_test), 
          callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir logs/fit